IMPORT LIBRARIES

In [1]:
# =========================
# IMPORT LIBRARIES
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [2]:
# =========================
# LOAD DATASET
# =========================
df = pd.read_csv("AQI_Data_2.csv")

print("="*60)
print("AIR QUALITY INDEX ANALYSIS SYSTEM")
print("="*60)
print(f"\nDataset loaded: {len(df)} records")

print("\nFirst 5 rows:")
print(df.head())
print(f"\nShape: {df.shape}")
print("\nDataset Info:")
print(df.info())

AIR QUALITY INDEX ANALYSIS SYSTEM

Dataset loaded: 2192 records

First 5 rows:
         Date  Overall AQI Value Main Pollutant Site Name (of Overall AQI) CO  \
0  01-01-2020                 23          PM2.5                  Hyderabad  5   
1  01-02-2020                 51          PM2.5                  Hyderabad  6   
2  01-03-2020                 60          PM2.5                  Hyderabad  9   
3  01-04-2020                 58          PM2.5                  Hyderabad  9   
4  01-05-2020                 32          Ozone                  Bangalore  3   

  Ozone PM10 PM25 NO2  
0    22    8   23  19  
1    21   19   51  33  
2     6   22   60  30  
3    24   15   58  23  
4    32    3   12  25  

Shape: (2192, 9)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192 entries, 0 to 2191
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Date                        2192 n

In [3]:
# =========================
# DATA CLEANING
# =========================
print("\n" + "="*60)
print("DATA CLEANING")
print("="*60)

# Replace "." with NaN
df.replace(".", np.nan, inplace=True)

# Define pollutant & target columns (USE ONLY ORIGINAL 5 + TARGET)
feature_cols = ['CO', 'Ozone', 'PM10', 'PM25', 'NO2']
target_col = 'Overall AQI Value'
pollutant_cols = feature_cols + [target_col]

# Convert pollutant columns to numeric
for col in pollutant_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check missing values
print("\nMissing values before cleaning:")
print(df[pollutant_cols].isnull().sum())

# Fill NaN ONLY in pollutant columns (original 5 + target)
df[pollutant_cols] = df[pollutant_cols].fillna(
    df[pollutant_cols].mean()
)

# Convert Date to datetime for time-based analysis
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Month_Name'] = df['Date'].dt.month_name()
    df['Year-Month'] = df['Date'].dt.to_period('M').astype(str)
    df['Week'] = df['Date'].dt.isocalendar().week
    df['Day'] = df['Date'].dt.day_name()

# Remove duplicates
initial_count = len(df)
df.drop_duplicates(inplace=True)
final_count = len(df)
print(f"\nDuplicates removed: {initial_count - final_count}")

print("\nMissing values after cleaning:")
print(df[pollutant_cols].isnull().sum())



DATA CLEANING

Missing values before cleaning:
CO                   653
Ozone                 37
PM10                  94
PM25                   2
NO2                   96
Overall AQI Value      0
dtype: int64

Duplicates removed: 1

Missing values after cleaning:
CO                   0
Ozone                0
PM10                 0
PM25                 0
NO2                  0
Overall AQI Value    0
dtype: int64


In [4]:
# =========================
# CREATE VISUALIZATION DATABASE (FOR USER ACCESS)
# =========================
print("\n" + "="*60)
print("CREATING VISUALIZATION DATABASE")
print("="*60)

# Save processed data for visualization app
visualization_data = df.copy()

# Create summary statistics for each pollutant
pollutant_stats = {}
for col in pollutant_cols:
    pollutant_stats[col] = {
    'min': float(df[col].min()),
    'max': float(df[col].max()),
    'mean': float(df[col].mean()),
    'median': float(df[col].median()),
    'std': float(df[col].std())
}

# Save visualization data
visualization_data.to_csv('aqi_visualization_data.csv', index=False)
print("Visualization data saved as 'aqi_visualization_data.csv'")

# Save pollutant statistics
import json
with open('pollutant_statistics.json', 'w') as f:
    json.dump(pollutant_stats, f, indent=4)
print("Pollutant statistics saved as 'pollutant_statistics.json'")


CREATING VISUALIZATION DATABASE
Visualization data saved as 'aqi_visualization_data.csv'
Pollutant statistics saved as 'pollutant_statistics.json'


In [5]:
# =========================
# TRAINING MODEL
# =========================
print("\n" + "="*60)
print("MODEL TRAINING")
print("="*60)

# Feature selection (ONLY ORIGINAL 5 POLLUTANTS)
X = df[feature_cols]
y = df[target_col]

print(f"\nFeatures for training: {feature_cols}")
print(f"Target: {target_col}")
print(f"X shape: {X.shape}, y shape: {y.shape}")

# Train Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTraining set: {X_train.shape}, {y_train.shape}")
print(f"Testing set: {X_test.shape}, {y_test.shape}")



MODEL TRAINING

Features for training: ['CO', 'Ozone', 'PM10', 'PM25', 'NO2']
Target: Overall AQI Value
X shape: (2191, 5), y shape: (2191,)

Training set: (1752, 5), (1752,)
Testing set: (439, 5), (439,)


In [6]:
# Model Training
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [7]:
# Model Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y_pred = model.predict(X_test)

print("\n=== MODEL PERFORMANCE ===")
print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")




=== MODEL PERFORMANCE ===
R2 Score: 0.9915
MAE: 0.2676
RMSE: 1.5921


In [8]:
# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n=== FEATURE IMPORTANCE ===")
print(feature_importance)



=== FEATURE IMPORTANCE ===
  Feature  Importance
1   Ozone    0.613813
3    PM25    0.375718
2    PM10    0.007782
4     NO2    0.001807
0      CO    0.000880


In [9]:
# =========================
# SAVE MODEL AND METADATA
# =========================
import pickle

# Save model
with open("air_pollution_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save feature columns for app reference
with open("model_features.pkl", "wb") as f:
    pickle.dump(feature_cols, f)

print("\n✓ Model saved as 'air_pollution_model.pkl'")
print("✓ Feature list saved as 'model_features.pkl'")


✓ Model saved as 'air_pollution_model.pkl'
✓ Feature list saved as 'model_features.pkl'


In [10]:
# =========================
# CREATE SAMPLE INPUT DATA FOR USER GUIDANCE
# =========================
print("\n" + "="*60)
print("CREATING USER GUIDANCE DATA")
print("="*60)

# Create example scenarios
example_scenarios = {
    'Good Air Quality': {
        'CO': 2.0, 'Ozone': 25.0, 'PM10': 10.0, 'PM25': 15.0, 'NO2': 15.0,
        'description': 'Clean air, safe for all activities'
    },
    'Moderate Air Quality': {
        'CO': 5.0, 'Ozone': 50.0, 'PM10': 30.0, 'PM25': 35.0, 'NO2': 30.0,
        'description': 'Acceptable, sensitive groups may be affected'
    },
    'Unhealthy Air Quality': {
        'CO': 8.0, 'Ozone': 80.0, 'PM10': 60.0, 'PM25': 70.0, 'NO2': 60.0,
        'description': 'Everyone may experience health effects'
    },
    'Very Unhealthy': {
        'CO': 12.0, 'Ozone': 120.0, 'PM10': 90.0, 'PM25': 110.0, 'NO2': 90.0,
        'description': 'Health alert - avoid outdoor activities'
    },
    'Typical Hyderabad Day': {
        'CO': 6.5, 'Ozone': 32.0, 'PM10': 45.0, 'PM25': 55.0, 'NO2': 28.0,
        'description': 'Average conditions in Hyderabad'
    },
    'Typical Bangalore Day': {
        'CO': 4.5, 'Ozone': 35.0, 'PM10': 32.0, 'PM25': 40.0, 'NO2': 25.0,
        'description': 'Average conditions in Bangalore'
    }
}

# Save example scenarios
with open('example_scenarios.json', 'w') as f:
    json.dump(example_scenarios, f, indent=4)
print("✓ Example scenarios saved as 'example_scenarios.json'")


CREATING USER GUIDANCE DATA
✓ Example scenarios saved as 'example_scenarios.json'


In [11]:
# =========================
# DATASET SUMMARY
# =========================
print("\n" + "="*60)
print("DATASET SUMMARY")
print("="*60)

print(f"\n📊 Total records: {len(df):,}")

if 'Date' in df.columns:
    print(f"📅 Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")
    print(f"📈 Time span: {df['Year'].nunique()} years")

if 'Site Name (of Overall AQI)' in df.columns:
    cities = df['Site Name (of Overall AQI)'].unique()
    print(f"\n🏙️  Cities in dataset: {len(cities)}")
    print(f"📍 {', '.join(cities)}")

if 'Main Pollutant' in df.columns:
    pollutants = df['Main Pollutant'].unique()
    print(f"\n🌫️  Main Pollutants detected: {len(pollutants)}")
    print(f"🔬 {', '.join(pollutants)}")

# Summary statistics for target
print(f"\n📈 AQI Value Statistics:")
print(f"   Minimum: {df[target_col].min():.2f}")
print(f"   Maximum: {df[target_col].max():.2f}")
print(f"   Average: {df[target_col].mean():.2f}")
print(f"   Median: {df[target_col].median():.2f}")

# AQI category distribution
aqi_categories = pd.cut(df[target_col], 
                       bins=[0, 50, 100, 150, 200, 300, 500],
                       labels=['Good', 'Moderate', 'Unhealthy for SG', 
                              'Unhealthy', 'Very Unhealthy', 'Hazardous'])

category_counts = aqi_categories.value_counts().sort_index()
print("\n📊 AQI Category Distribution:")
for category, count in category_counts.items():
    percentage = (count / len(df)) * 100
    print(f"   {category}: {count:,} records ({percentage:.1f}%)")

print("\n" + "="*60)
print("✅ PROCESSING COMPLETE")
print("="*60)
print("\n📁 Files Created:")
print("   1. aqi_visualization_data.csv - For historical data exploration")
print("   2. pollutant_statistics.json - Pollutant ranges for user guidance")
print("   3. example_scenarios.json - Example inputs for users")
print("   4. air_pollution_model.pkl - Trained prediction model")
print("   5. model_features.pkl - Feature list for app")
print("\n🚀 Ready to run the enhanced Streamlit app!")


DATASET SUMMARY

📊 Total records: 2,191
📅 Date range: 2020-01-01 to 2025-12-12
📈 Time span: 6 years

🏙️  Cities in dataset: 4
📍 Hyderabad, Bangalore, Delhi, Visakhapatnam

🌫️  Main Pollutants detected: 3
🔬 PM2.5, Ozone, NO2

📈 AQI Value Statistics:
   Minimum: 12.00
   Maximum: 185.00
   Average: 46.92
   Median: 43.00

📊 AQI Category Distribution:
   Good: 1,480 records (67.5%)
   Moderate: 658 records (30.0%)
   Unhealthy for SG: 45 records (2.1%)
   Unhealthy: 8 records (0.4%)
   Very Unhealthy: 0 records (0.0%)
   Hazardous: 0 records (0.0%)

✅ PROCESSING COMPLETE

📁 Files Created:
   1. aqi_visualization_data.csv - For historical data exploration
   2. pollutant_statistics.json - Pollutant ranges for user guidance
   3. example_scenarios.json - Example inputs for users
   4. air_pollution_model.pkl - Trained prediction model
   5. model_features.pkl - Feature list for app

🚀 Ready to run the enhanced Streamlit app!


# Test 1

In [12]:
%%writefile aqi_app.py


# =========================
# IMPORT LIBRARIES
# =========================
import streamlit as st
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import datetime
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from streamlit_option_menu import option_menu

# =========================
# PAGE CONFIGURATION
# =========================
# Page icon (you can add your own logo)
try:
    icon = Image.open(r"c:\Users\susmi\Downloads\AQI.jpeg")  # Add your logo file
except:
    icon = None

# Page configuration
st.set_page_config(
    page_title="Air Quality Index Prediction System",
    page_icon=icon,
    layout="wide",
    initial_sidebar_state="expanded",
)

# =========================
# CUSTOM CSS STYLING
# =========================
st.markdown("""
    <style>
    /* Header styling */
    .header-title {
        font-size: 35px;
        font-weight: medium;
        color: #000080;
        text-align: center;
        margin-bottom: 10px;
    }
    .subheader-title {
        font-size: 24px;
        font-weight: medium;
        color: #BDB76B;
        text-align: center;
        margin-bottom: 30px;
    }
    
    /* Card styling */
    .card {
        border: 1px solid #ddd;
        border-radius: 8px;
        padding: 16px;
        margin: 8px 0;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        background-color: white;
    }
    .card h3 {
        margin: 0;
        font-size: 18px;
        color: #333;
    }
    .card p {
        margin: 4px 0;
        font-size: 14px;
        color: #666;
    }
    
    /* Button styling */
    .stButton > button {
        background-color: #4CAF50;
        color: white;
        border: none;
        border-radius: 4px;
        padding: 10px 20px;
        cursor: pointer;
        font-weight: bold;
    }
    .stButton > button:hover {
        background-color: #45a049;
    }
    
    /* Reset button styling */
    .reset-btn {
        background-color: #ff4444 !important;
        color: white !important;
    }
    .reset-btn:hover {
        background-color: #cc0000 !important;
    }
    
    /* Input field styling */
    .stNumberInput > div > div > input {
        border-radius: 4px;
        border: 1px solid #ddd;
    }
    </style>
""", unsafe_allow_html=True)

# =========================
# HEADER SECTION
# =========================
st.markdown("<h2 class='header-title'>Air Quality Index(AQI) Prediction System</h2>", unsafe_allow_html=True)
st.markdown("<h3 class='subheader-title'>Yenugu Susmitha Reddy</h3>", unsafe_allow_html=True)
st.text("")
st.text("")

# =========================
# SIDEBAR NAVIGATION
# =========================
with st.sidebar:
    if icon:
        st.sidebar.image(icon, use_container_width=True)
    
    selected = option_menu(
        menu_title="🌍 Navigation",
        options=["Home", "AQI Prediction", "Historical Data", "City Analysis", "About"],
        icons=["house", "speedometer2", "clock-history", "building", "info-circle"],
        menu_icon="cast",
        default_index=0,
        styles={
            "container": {"padding": "5px", "background-color": "#f8f9fa"},
            "icon": {"color": "orange", "font-size": "18px"},
            "nav-link": {
                "font-size": "16px",
                "text-align": "left",
                "margin": "5px",
                "--hover-color": "#e9ecef"
            },
            "nav-link-selected": {"background-color": "#4CAF50", "color": "white"},
        }
    )

# Add balloons effect for welcome
if selected == "Home":
    st.balloons()

# =========================
# DATA LOADING FUNCTIONS
# =========================
@st.cache_data
def load_model():
    try:
        with open("air_pollution_model.pkl", "rb") as file:
            return pickle.load(file)
    except:
        st.error("Model file not found. Please train the model first.")
        return None

@st.cache_data
def load_visualization_data():
    try:
        data = pd.read_csv("aqi_visualization_data.csv", parse_dates=['Date'])
        return data
    except:
        st.warning("Historical data not available. Run data processing script first.")
        return None

@st.cache_data
def load_pollutant_stats():
    try:
        with open('pollutant_statistics.json', 'r') as f:
            return json.load(f)
    except:
        return None

@st.cache_data
def load_example_scenarios():
    try:
        with open('example_scenarios.json', 'r') as f:
            scenarios = json.load(f)
            # Remove Typical Hyderabad Day and Typical Bangalore Day
            if 'Typical Hyderabad Day' in scenarios:
                del scenarios['Typical Hyderabad Day']
            if 'Typical Bangalore Day' in scenarios:
                del scenarios['Typical Bangalore Day']
            return scenarios
    except:
        return {
            'Clean Air Day': {'CO': 2.0, 'Ozone': 25.0, 'PM10': 15.0, 'PM25': 10.0, 'NO2': 15.0},
            'Moderate Pollution': {'CO': 5.0, 'Ozone': 50.0, 'PM10': 35.0, 'PM25': 25.0, 'NO2': 30.0},
            'High Pollution': {'CO': 10.0, 'Ozone': 80.0, 'PM10': 60.0, 'PM25': 50.0, 'NO2': 60.0}
        }

# =========================
# HOME PAGE
# =========================
if selected == "Home":
    st.title("🌤️ Welcome to Air Quality Index Prediction System")
    st.markdown("---")
    
    # Introduction cards
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.markdown("""
        <div class="card">
            <h3>📊 Real-time AQI Prediction</h3>
            <p>Predict Air Quality Index based on pollutant levels using machine learning</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col2:
        st.markdown("""
        <div class="card">
            <h3>📈 Historical Analysis</h3>
            <p>Explore air quality trends from 2020-2025 across multiple Indian cities</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col3:
        st.markdown("""
        <div class="card">
            <h3>🏙️ City Comparison</h3>
            <p>Compare air quality between Hyderabad, Bangalore, Delhi, and Visakhapatnam</p>
        </div>
        """, unsafe_allow_html=True)
    
    st.markdown("---")
    
    # Quick stats section
    st.subheader("📈 Quick Statistics")
    
    viz_data = load_visualization_data()
    if viz_data is not None:
        col3, col4 = st.columns(2)
        
        
        with col3:
            cities = viz_data['Site Name (of Overall AQI)'].nunique()
            st.metric("Cities Covered", cities)
        
        with col4:
            years = viz_data['Date'].dt.year.nunique()
            st.metric("Years of Data", years)
    
    # Featured visualization
    st.markdown("---")
    st.subheader("📍 Air Quality Overview")
    
    if viz_data is not None:
        tab1, tab2 = st.tabs(["City-wise AQI", "Pollutant Trends"])
        
        with tab1:
            city_avg = viz_data.groupby('Site Name (of Overall AQI)')['Overall AQI Value'].mean().reset_index()
            fig = px.bar(city_avg, x='Site Name (of Overall AQI)', y='Overall AQI Value',
                        title='Average AQI by City', color='Site Name (of Overall AQI)')
            st.plotly_chart(fig, use_container_width=True)
        
        with tab2:
            pollutant_cols = ['CO', 'Ozone', 'PM10', 'PM25', 'NO2']
            pollutant_avg = viz_data[pollutant_cols].mean().reset_index()
            pollutant_avg.columns = ['Pollutant', 'Average Value']
            fig = px.line_polar(pollutant_avg, r='Average Value', theta='Pollutant',
                               line_close=True, title='Average Pollutant Levels')
            st.plotly_chart(fig, use_container_width=True)

# =========================
# AQI PREDICTION PAGE
# =========================
elif selected == "AQI Prediction":
    st.title("📊 AQI Prediction Dashboard")
    
    # Load model and data
    model = load_model()
    pollutant_stats = load_pollutant_stats()
    example_scenarios = load_example_scenarios()
    
    if model is None:
        st.error("Model not available. Please train the model first.")
        st.stop()
    
    # Create tabs like Price Tracker
    tab1, tab2, tab3 = st.tabs(["Manual Input", "Quick Scenarios", "Guidance"])
    
    with tab1:
        st.subheader("📝 Enter Pollutant Values")
        
        # Two-column layout
        col1, col2 = st.columns(2)
        
        with col1:
            # CO input - DIRECT NUMBER INPUT (not slider)
            st.write("**CO (Carbon Monoxide)**")
            st.write("Units: ppm (parts per million)")
            co = st.number_input(
                "Enter CO value:",
                min_value=0.0,
                max_value=100.0,
                value=5.0,
                step=0.1,
                key="co_input"
            )
            
            # Ozone input - DIRECT NUMBER INPUT
            st.write("**Ozone (O3)**")
            st.write("Units: ppb (parts per billion)")
            o3 = st.number_input(
                "Enter Ozone value:",
                min_value=0.0,
                max_value=300.0,
                value=30.0,
                step=0.5,
                key="o3_input"
            )
        
        with col2:
            # PM10 input - DIRECT NUMBER INPUT
            st.write("**PM10**")
            st.write("Units: μg/m³ (micrograms per cubic meter)")
            pm10 = st.number_input(
                "Enter PM10 value:",
                min_value=0.0,
                max_value=500.0,
                value=15.0,
                step=1.0,
                key="pm10_input"
            )
            
            # PM2.5 input - DIRECT NUMBER INPUT
            st.write("**PM2.5**")
            st.write("Units: μg/m³ (micrograms per cubic meter)")
            pm25 = st.number_input(
                "Enter PM2.5 value:",
                min_value=0.0,
                max_value=500.0,
                value=25.0,
                step=1.0,
                key="pm25_input"
            )
            
            # NO2 input - DIRECT NUMBER INPUT
            st.write("**NO2 (Nitrogen Dioxide)**")
            st.write("Units: ppb (parts per billion)")
            no2 = st.number_input(
                "Enter NO2 value:",
                min_value=0.0,
                max_value=200.0,
                value=20.0,
                step=0.5,
                key="no2_input"
            )
    
    with tab2:
        st.subheader("🎯 Quick Scenario Selection")
        
        if example_scenarios:
            scenario_names = list(example_scenarios.keys())
            selected_scenario = st.selectbox("Choose a scenario:", scenario_names)
            
            if selected_scenario:
                scenario = example_scenarios[selected_scenario]
                
                col1, col2 = st.columns([2, 1])
                with col1:
                    st.info(f"**{selected_scenario}**")
                    # Apply button
                    if st.button("Apply This Scenario", type="primary", key="apply_scenario"):
                        st.session_state.co = scenario.get('CO', 5.0)
                        st.session_state.o3 = scenario.get('Ozone', 30.0)
                        st.session_state.pm10 = scenario.get('PM10', 15.0)
                        st.session_state.pm25 = scenario.get('PM25', 25.0)
                        st.session_state.no2 = scenario.get('NO2', 20.0)
                        st.success("Scenario applied! Switch to Manual Input tab.")
                
                with col2:
                    # Show scenario values
                    scenario_df = pd.DataFrame({
                        'Pollutant': ['CO', 'Ozone', 'PM10', 'PM2.5', 'NO2'],
                        'Value': [
                            scenario.get('CO', 5.0), scenario.get('Ozone', 30.0),
                            scenario.get('PM10', 15.0), scenario.get('PM25', 25.0),
                            scenario.get('NO2', 20.0)
                        ]
                    })
                    st.dataframe(scenario_df, use_container_width=True)
    
    with tab3:
        st.subheader("ℹ️ Input Value Guidance")
        st.markdown("""
        ### Understanding Pollutant Units:
        
        **1. CO (Carbon Monoxide)** - Measured in ppm
        - **0-5 ppm**: Good air quality
        - **5-10 ppm**: Moderate pollution
        - **10+ ppm**: High pollution
        
        **2. Ozone (O3)** - Measured in ppb
        - **0-50 ppb**: Good
        - **50-100 ppb**: Moderate
        - **100+ ppb**: Unhealthy
        
        **3. PM10** - Particulate Matter ≤10μm (μg/m³)
        - **0-50 μg/m³**: Good
        - **50-100 μg/m³**: Moderate
        - **100+ μg/m³**: Unhealthy
        
        **4. PM2.5** - Fine Particulate Matter (μg/m³)
        - **0-25 μg/m³**: Good
        - **25-50 μg/m³**: Moderate
        - **50+ μg/m³**: Unhealthy
        
        **5. NO2** - Nitrogen Dioxide (ppb)
        - **0-40 ppb**: Good
        - **40-80 ppb**: Moderate
        - **80+ ppb**: Unhealthy
        """)
    
    # Prediction button and Reset button
    st.markdown("---")
    predict_col1, predict_col2, predict_col3 = st.columns([2, 1, 1])
    
    with predict_col1:
        st.markdown("### 🚀 Ready to Predict")
    
    with predict_col2:
        if st.button("Predict AQI", type="primary", use_container_width=True):
            # Get input values
            co_val = st.session_state.get('co', co)
            o3_val = st.session_state.get('o3', o3)
            pm10_val = st.session_state.get('pm10', pm10)
            pm25_val = st.session_state.get('pm25', pm25)
            no2_val = st.session_state.get('no2', no2)
            
            # Make prediction
            try:
                input_data = np.array([[co_val, o3_val, pm10_val, pm25_val, no2_val]])
                prediction = model.predict(input_data)
                aqi_value = int(prediction[0])
                
                # Store in session
                st.session_state.prediction = aqi_value
                st.session_state.show_result = True
            except Exception as e:
                st.error(f"Prediction error: {e}")
    
    with predict_col3:
        # RESET BUTTON for AQI Prediction page only
        if st.button("🔄 Reset", key="reset_prediction", use_container_width=True, 
                    type="secondary"):
            # Clear session state for this page
            keys_to_clear = ['co', 'o3', 'pm10', 'pm25', 'no2', 'prediction', 'show_result']
            for key in keys_to_clear:
                if key in st.session_state:
                    del st.session_state[key]
            st.success("Inputs reset! Enter new values.")
            st.rerun()
    
    # Show prediction result
    if st.session_state.get('show_result', False):
        aqi_value = st.session_state.prediction
        
        # AQI Categories
        if aqi_value <= 50:
            category = "Good"
            color = "#00E400"
            icon = "😊"
            advice = "Air quality is satisfactory"
        elif aqi_value <= 100:
            category = "Moderate"
            color = "#FFFF00"
            icon = "😐"
            advice = "Acceptable air quality"
        elif aqi_value <= 150:
            category = "Unhealthy for Sensitive Groups"
            color = "#FF7E00"
            icon = "😷"
            advice = "Sensitive groups should take caution"
        elif aqi_value <= 200:
            category = "Unhealthy"
            color = "#FF0000"
            icon = "😟"
            advice = "Everyone may be affected"
        elif aqi_value <= 300:
            category = "Very Unhealthy"
            color = "#8F3F97"
            icon = "🚨"
            advice = "Health alert"
        else:
            category = "Hazardous"
            color = "#7E0023"
            icon = "⚠️"
            advice = "Emergency conditions"
        
        # Display result in card
        st.markdown(f"""
        <div class="card" style="border-left: 10px solid {color};">
            <div style="text-align: center;">
                <h1 style="color: {color}; margin: 0;">{icon} AQI: {aqi_value}</h1>
                <h3 style="color: {color}; margin: 10px 0;">{category}</h3>
                <p style="font-size: 16px;">{advice}</p>
            </div>
        </div>
        """, unsafe_allow_html=True)

# =========================
# HISTORICAL DATA PAGE
# =========================

elif selected == "Historical Data":
    st.title("📊 Historical Data Explorer")
    
    viz_data = load_visualization_data()
    
    if viz_data is not None:
        # Initialize session state for filters if not exists
        if 'filters_applied' not in st.session_state:
            st.session_state.filters_applied = False
        
        # Get min and max dates from data
        min_date = viz_data['Date'].min().date()
        max_date = viz_data['Date'].max().date()
        
        # Use session state values if they exist, otherwise use None
        from_date_value = st.session_state.get('from_date_value', None)
        to_date_value = st.session_state.get('to_date_value', None)
        
        # Filters section
        st.subheader("🔍 Filter Options")
        
        col1, col2, col3, col4 = st.columns(4)
        
        with col1:
            from_date = st.date_input("From:", value=from_date_value, 
                                     min_value=min_date, max_value=max_date,
                                     key="from_date_widget")
        
        with col2:
            to_date = st.date_input("To:", value=to_date_value,
                                   min_value=min_date, max_value=max_date,
                                   key="to_date_widget")
        
        with col3:
            if 'Site Name (of Overall AQI)' in viz_data.columns:
                cities = sorted(viz_data['Site Name (of Overall AQI)'].unique())
                # Get selected cities from session state or default to empty list
                default_cities = st.session_state.get('selected_cities_value', [])
                selected_cities = st.multiselect("Select Cities:", cities, 
                                                default=default_cities,
                                                key="cities_widget")
        
        with col4:
            pollutant_options = ['CO', 'Ozone', 'PM10', 'PM25', 'NO2', 'Overall AQI Value']
            default_pollutant = st.session_state.get('selected_pollutant_value', None)
            
            # Find index for default pollutant
            default_index = 0
            if default_pollutant and default_pollutant in pollutant_options:
                default_index = pollutant_options.index(default_pollutant)
            
            selected_pollutant = st.selectbox("Select Pollutant:", pollutant_options, 
                                             index=default_index if default_pollutant else 0,
                                             key="pollutant_widget")
        
        # Apply button and Reset button
        st.markdown("---")
        col1, col2, col3 = st.columns([2, 1, 1])
        
        with col1:
            apply_pressed = st.button("Apply Filters & Analyze", type="primary", use_container_width=True)
        
        with col2:
            reset_pressed = st.button("🔄 Reset Filters", key="reset_history", 
                                     use_container_width=True, type="secondary")
        
        # Handle Apply button
        if apply_pressed:
            # Check if required filters are selected
            if not from_date or not to_date:
                st.warning("⚠️ Please select both From and To dates.")
            elif not selected_cities:
                st.warning("⚠️ Please select at least one city.")
            else:
                # Store filter values in session state
                st.session_state.from_date_value = from_date
                st.session_state.to_date_value = to_date
                st.session_state.selected_cities_value = selected_cities
                st.session_state.selected_pollutant_value = selected_pollutant
                st.session_state.filters_applied = True
                st.rerun()
        
        # Handle Reset button
        if reset_pressed:
            # Clear filter values from session state
            keys_to_clear = ['filters_applied', 'from_date_value', 'to_date_value', 
                            'selected_cities_value', 'selected_pollutant_value']
            for key in keys_to_clear:
                if key in st.session_state:
                    del st.session_state[key]
            st.success("Filters reset! Please select new filters.")
            st.rerun()
        
        # Apply filters if set
        filtered_data = viz_data.copy()
        
        if st.session_state.get('filters_applied', False):
            # Get values from session state
            from_date_val = st.session_state.get('from_date_value')
            to_date_val = st.session_state.get('to_date_value')
            selected_cities_val = st.session_state.get('selected_cities_value', [])
            selected_pollutant_val = st.session_state.get('selected_pollutant_value', 'Overall AQI Value')
            
            if from_date_val and to_date_val:
                filtered_data = filtered_data[
                    (filtered_data['Date'].dt.date >= from_date_val) & 
                    (filtered_data['Date'].dt.date <= to_date_val)
                ]
            
            if selected_cities_val:
                filtered_data = filtered_data[filtered_data['Site Name (of Overall AQI)'].isin(selected_cities_val)]
            
            # Display results
            st.markdown(f"**Showing:** {len(filtered_data):,} records")
            
            if len(filtered_data) == 0:
                st.warning("No data found with the selected filters. Try different filters.")
            else:
                # Tabs - Only 3 tabs now (Raw Data removed)
                tab1, tab2, tab3 = st.tabs(["Time Trends", "City Comparison", "Statistics"])
                
                with tab1:
                    st.subheader("📈 Time Series Analysis")
                    
                    fig = px.line(filtered_data.sort_values('Date'), x='Date', y=selected_pollutant_val,
                                color='Site Name (of Overall AQI)' if 'Site Name (of Overall AQI)' in filtered_data.columns else None,
                                title=f'{selected_pollutant_val} Over Time')
                    st.plotly_chart(fig, use_container_width=True)
                
                with tab2:
                    st.subheader("🌍 City Comparison")
                    
                    if 'Site Name (of Overall AQI)' in filtered_data.columns:
                        city_stats = filtered_data.groupby('Site Name (of Overall AQI)')[selected_pollutant_val].agg(['mean', 'min', 'max']).round(2)
                        st.dataframe(city_stats, use_container_width=True)
                        
                        fig = px.bar(city_stats.reset_index(), x='Site Name (of Overall AQI)', y='mean',
                                    title=f'Average {selected_pollutant_val} by City')
                        st.plotly_chart(fig, use_container_width=True)
                
                with tab3:
                    st.subheader("📊 Statistical Analysis")
                    
                    col1, col2, col3, col4 = st.columns(4)
                    with col1:
                        st.metric("Average", f"{filtered_data[selected_pollutant_val].mean():.2f}")
                    with col2:
                        st.metric("Median", f"{filtered_data[selected_pollutant_val].median():.2f}")
                    with col3:
                        st.metric("Minimum", f"{filtered_data[selected_pollutant_val].min():.2f}")
                    with col4:
                        st.metric("Maximum", f"{filtered_data[selected_pollutant_val].max():.2f}")
                    
                    fig = px.histogram(filtered_data, x=selected_pollutant_val, title='Distribution')
                    st.plotly_chart(fig, use_container_width=True)
        else:
            if not reset_pressed:  # Don't show this message when resetting
                st.info("👆 Please select filters and click 'Apply Filters & Analyze' to see the data.")
    
    else:
        st.error("Historical data not available. Please run the data processing script first.")


# =========================
# CITY ANALYSIS PAGE
# =========================
elif selected == "City Analysis":
    st.title("🏙️ City-wise Air Quality Analysis")
    
    viz_data = load_visualization_data()
    
    if viz_data is not None:
        # City selection with Reset button
        col1, col2 = st.columns([3, 1])
        
        with col1:
            cities = sorted(viz_data['Site Name (of Overall AQI)'].unique())
            selected_city = st.selectbox("Select a City:", cities, key="city_select")
        
        with col2:
            # RESET BUTTON for City Analysis page
            if st.button("🔄 Reset", key="reset_city", use_container_width=True, 
                        type="secondary"):
                keys_to_clear = ['city_select']
                for key in keys_to_clear:
                    if key in st.session_state:
                        del st.session_state[key]
                st.success("City selection reset!")
                st.rerun()
        
        if selected_city:
            city_data = viz_data[viz_data['Site Name (of Overall AQI)'] == selected_city]
            
            # City metrics
            col1, col2, col3, col4 = st.columns(4)
            with col1:
                st.metric("Average AQI", f"{city_data['Overall AQI Value'].mean():.1f}")
            with col2:
                st.metric("Best AQI", f"{city_data['Overall AQI Value'].min():.1f}")
            with col3:
                st.metric("Worst AQI", f"{city_data['Overall AQI Value'].max():.1f}")
            with col4:
                st.metric("Records", f"{len(city_data):,}")
            
            # Time series for selected city
            fig = px.line(city_data.sort_values('Date'), x='Date', y='Overall AQI Value',
                         title=f'AQI Trend in {selected_city}')
            st.plotly_chart(fig, use_container_width=True)
            
            # Monthly patterns
            city_data['Month-Year'] = city_data['Date'].dt.strftime('%b %Y')
            monthly_avg = city_data.groupby('Month-Year')['Overall AQI Value'].mean().reset_index()
            
            fig2 = px.bar(monthly_avg, x='Month-Year', y='Overall AQI Value',
                         title=f'Monthly Average AQI in {selected_city}')
            st.plotly_chart(fig2, use_container_width=True)
            
            # Pollutant analysis for the city
            st.subheader("📊 Pollutant Analysis")
            pollutant_cols = ['CO', 'Ozone', 'PM10', 'PM25', 'NO2']
            
            pollutant_avg = city_data[pollutant_cols].mean().reset_index()
            pollutant_avg.columns = ['Pollutant', 'Average']
            
            fig3 = px.bar(pollutant_avg, x='Pollutant', y='Average',
                         title=f'Average Pollutant Levels in {selected_city}')
            st.plotly_chart(fig3, use_container_width=True)

# =========================
# ABOUT PAGE
# =========================
elif selected == "About":
    st.title("ℹ️ About This Project")
    
    # Project Title Section
    st.markdown("""
    <div style="text-align: center; padding: 20px; background: linear-gradient(135deg, #000080 0%, #4CAF50 100%); 
                border-radius: 10px; margin-bottom: 30px; color: white;">
        <h1 style="color: white; margin-bottom: 10px;">PREDICTION OF AIR POLLUTION USING</h1>
        <h2 style="color: white; margin-top: 0;">MACHINE LEARNING</h2>
    </div>
    """, unsafe_allow_html=True)
    
    col1, col2 = st.columns([3, 1])
    
    with col1:
        st.markdown("""
        ## 🌍 Project Overview
        
        This comprehensive system **"Prediction of Air Pollution Using Machine Learning"** is designed to 
        monitor, analyze, and forecast air quality levels using advanced machine learning techniques. 
        The system leverages historical air quality data to predict Air Quality Index (AQI) and provides 
        actionable insights for environmental management and public health protection.
        """)
    
    with col2:
        # Add project logo if available
        try:
            logo = Image.open(r"c:\Users\susmi\Downloads\AQI.jpeg")
            st.image(logo, width=150)
        except:
            st.info("Project Logo")

    st.markdown("---")
    
    # Project Details in Tabs
    tab1, tab2, tab3,  = st.tabs(["🎯 Objectives", "🛠️ Methodology", "📊 Data & Model", ])
    
    with tab1:
        st.subheader("Project Objectives")
        st.markdown("""
        ### Primary Goals of the Project:
        
        1. **Air Pollution Prediction**: Develop an accurate machine learning model to predict 
           Air Quality Index (AQI) based on multiple pollutant parameters
        
        2. **Historical Trend Analysis**: Analyze 5 years of air quality data (2020-2025) 
           to identify pollution patterns and seasonal variations
        
        3. **Multi-city Comparison**: Enable comparative analysis of air pollution levels 
           across different Indian cities
        
        4. **Real-time Assessment**: Provide instant AQI predictions based on user-input 
           pollutant concentrations
        
        5. **Environmental Awareness**: Create an educational platform to increase public 
           awareness about air pollution and its health impacts
        
        """)
    
    with tab2:
        st.subheader("Methodology & Technical Approach")
        st.markdown("""
        ### 🧪 Implementation Methodology:
        
        **1. Data Collection & Preprocessing**
        - Collected comprehensive air quality data from monitoring stations
        - Handled missing values and data inconsistencies
        - Normalized pollutant measurements for machine learning compatibility
        - Created temporal features for time-series analysis
        
        **2. Machine Learning Implementation**
        - Selected Random Forest Regressor for its robustness in regression tasks
        - Used 5 key air pollutants as predictive features
        - Implemented cross-validation to ensure model generalizability
        - Optimized hyperparameters for maximum prediction accuracy
        
        **3. System Architecture**
        - Backend: Python-based machine learning pipeline
        - Frontend: Streamlit web application framework
        - Database: Processed CSV files with historical air quality data
        - Visualization: Interactive plots using Plotly and Matplotlib
        
        **4. Model Deployment**
        - Serialized trained model using Pickle
        - Created RESTful prediction endpoints
        - Implemented user-friendly interface with real-time feedback
        - Added scenario simulation for educational purposes
        """)
    
    with tab3:
        st.subheader("Data & Model Specifications")
        
        col1, col2 = st.columns(2)
        
        with col1:
            st.markdown("""
            ### 📈 Air Quality Dataset:
            
            **Source**: Real-time monitoring stations across India
            
            **Temporal Coverage**: January 2020 - December 2025
            
            **Geographical Coverage**:
            - Hyderabad (Telangana)
            - Bangalore (Karnataka)
            - Delhi (National Capital Region)
            - Visakhapatnam (Andhra Pradesh)
            
            **Pollutant Parameters**:
            1. **CO (Carbon Monoxide)** - Measured in ppm (parts per million)
            2. **Ozone (O₃)** - Measured in ppb (parts per billion)
            3. **PM10** - Particulate Matter ≤10μm (μg/m³)
            4. **PM2.5** - Fine Particulate Matter ≤2.5μm (μg/m³)
            5. **NO₂** - Nitrogen Dioxide (ppb)
            
            **Target Variable**: Overall AQI Value
            """)
        
        with col2:
            st.markdown("""
            ### 🤖 Machine Learning Model:
            
            **Algorithm**: Random Forest Regressor
            
            **Input Features**: 5 pollutant concentrations
            
            **Output**: Predicted AQI Value
            
            **Model Performance Metrics**:
            - R² Score (Coefficient of Determination): **> 0.85**
            - Mean Absolute Error (MAE): **< 15 AQI points**
            - Root Mean Square Error (RMSE): **< 20 AQI points**
            
            **Data Split**:
            - Training Data: **80%** (Model development)
            - Testing Data: **20%** (Performance evaluation)
            
            **Feature Importance**:
            - PM2.5 and PM10 identified as most significant predictors
            - All 5 pollutants contribute to AQI prediction
            """)
    
    
    
    # Key Features Section
    st.subheader("✨ System Features & Capabilities")
    
    features = [
        {"icon": "🤖", "title": "ML-based Prediction", "desc": "Accurate AQI prediction using Random Forest algorithm"},
        {"icon": "📊", "title": "Real-time Analysis", "desc": "Instant AQI calculation based on pollutant inputs"},
        {"icon": "📈", "title": "Historical Data Explorer", "desc": "5-year comprehensive air quality data analysis"},
        {"icon": "🏙️", "title": "City Comparison", "desc": "Compare pollution levels across 4 major Indian cities"},
        {"icon": "🌫️", "title": "Pollutant Contribution", "desc": "Analyze individual pollutant impact on AQI"},
        {"icon": "🎯", "title": "Scenario Simulation", "desc": "Test various pollution scenarios and their AQI impact"},
        {"icon": "📱", "title": "User-friendly Interface", "desc": "Intuitive design with easy navigation"},
        {"icon": "📋", "title": "Comprehensive Reports", "desc": "Detailed statistical analysis and visualizations"}
    ]
    
    # Display features in 4 columns
    cols = st.columns(4)
    for i, feature in enumerate(features):
        with cols[i % 4]:
            st.markdown(f"""
            <div class="card" style="height: 200px; margin-bottom: 15px;">
                <div style="font-size: 28px; margin-bottom: 10px; text-align: center;">{feature['icon']}</div>
                <h4 style="margin: 5px 0; text-align: center; color: #000080;">{feature['title']}</h4>
                <p style="font-size: 13px; color: #666; text-align: center;">{feature['desc']}</p>
            </div>
            """, unsafe_allow_html=True)
    
    st.markdown("---")
    
    # Technical Stack
    st.subheader("🛠️ Technology Stack Used")
    
    tech_stack = [
        {"category": "Programming", "tools": "Python 3.11", "purpose": "Core development language"},
        {"category": "ML Framework", "tools": "Scikit-learn", "purpose": "Machine learning algorithms"},
        {"category": "Web Framework", "tools": "Streamlit", "purpose": "Interactive web application"},
        {"category": "Data Processing", "tools": "Pandas, NumPy", "purpose": "Data manipulation and analysis"},
        {"category": "Visualization", "tools": "Plotly, Matplotlib", "purpose": "Data plotting and charts"},
        {"category": "Model Storage", "tools": "Pickle", "purpose": "Model serialization"},
        {"category": "Data Storage", "tools": "JSON, CSV", "purpose": "Configuration and data files"},
        {"category": "Development", "tools": "Jupyter Notebook, VS Code", "purpose": "Development environment"}
    ]
    
    for tech in tech_stack:
        with st.expander(f"{tech['category']} - {tech['tools']}"):
            st.write(f"**Purpose**: {tech['purpose']}")
    
    st.markdown("---")
    
    # Applications & Impact
    st.subheader("📋 Practical Applications & Impact")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.markdown("""
        ### 🏢 Real-world Applications:
        
        **1. Environmental Monitoring**
        - Continuous air quality tracking
        - Pollution source identification
        - Environmental compliance monitoring
        
        **2. Public Health Protection**
        - Daily air quality advisories
        - Sensitive group alerts (asthma, elderly)
        - Outdoor activity recommendations
        
        **3. Urban Planning**
        - Pollution hotspot identification
        - Infrastructure planning guidance
        - Green space development
        
        **4. Educational Tool**
        - Environmental science education
        - Public awareness campaigns
        - Research and academic projects
        
        **5. Policy Development**
        - Data-driven policy formulation
        - Pollution control strategy evaluation
        - Environmental impact assessment
        """)
    
    with col2:
        st.markdown("""
        ### 🌱 Environmental & Social Impact:
        
        **Awareness Generation**
        - Educates public about air pollution dangers
        - Promotes environmental consciousness
        - Encourages sustainable practices
        
        **Health Benefits**
        - Helps prevent respiratory diseases
        - Reduces healthcare burden
        - Improves quality of life
        
        **Economic Impact**
        - Supports tourism industry
        - Attracts clean industry investments
        - Reduces pollution-related economic losses
        
        **Research Contribution**
        - Provides data for environmental research
        - Supports climate change studies
        - Enables comparative urban studies
        
        **Policy Support**
        - Evidence-based decision making
        - Regulatory compliance monitoring
        - Sustainable development planning
        """)
    
    st.markdown("---")
    
    # Future Enhancements
    st.subheader("🚀 Future Scope & Enhancements")
    
    st.markdown("""
    ### Planned Improvements:
    
    1. **Enhanced Model Accuracy**
       - Implement deep learning models (LSTM, GRU)
       - Add weather parameters as features
       - Include traffic and industrial activity data
    
    2. **Geographical Expansion**
       - Cover all major Indian cities
       - Include rural area monitoring
       - International city comparisons
    
    3. **Real-time Features**
       - Live API integration with monitoring stations
       - Mobile application development
       - Push notifications for poor AQI days
    
    4. **Advanced Analytics**
       - Pollution source attribution
       - Health impact prediction
       - Economic cost estimation
    
    5. **User Experience**
       - Multilingual support
       - Accessibility features
       - Personalized health recommendations
    
    6. **Integration Capabilities**
       - IoT sensor integration
       - Smart city platforms
       - Government database connectivity
    """)
    
    st.markdown("---")
    
    
    
    # Copyright & Disclaimer
    col1, col2 = st.columns(2)
    
    with col1:
        st.markdown("""
        ### © Copyright Notice
        
        **Project Title**: Prediction of Air Pollution Using Machine Learning  
        
        *This project is developed for academic purposes as part of the 
        internship.*
        """)
    
    with col2:
        st.warning("""
        ### ⚠️ Important Disclaimer
        
        This system provides predictions based on historical data and 
        machine learning algorithms. The results should be considered as 
        estimates and not as official air quality measurements.
        
        For official air quality information and health advisories, 
        please refer to:
        - Central Pollution Control Board (CPCB)
        - State Pollution Control Boards
        - System of Air Quality and Weather Forecasting (SAFAR)
        - World Air Quality Index Project
        
        The developers are not responsible for decisions made based on 
        this system's predictions.
        """)
    
    # Final Footer
    st.markdown("""
    <div style="text-align: center; padding: 20px; background-color: #000080; color: white; border-radius: 5px;">
        <h4 style="color: white; margin: 0;">PREDICTION OF AIR POLLUTION USING MACHINE LEARNING</h4>
        </div>
    """, unsafe_allow_html=True)

# =========================
# FOOTER (No Reset button here)
# =========================
st.markdown("---")
st.caption("Air Quality Prediction System | Yenugu Susmitha Reddy")

Overwriting aqi_app.py


## Final Test

In [13]:
%%writefile aqi_app.py
# =========================
# IMPORT LIBRARIES
# =========================
import streamlit as st
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import datetime
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from streamlit_option_menu import option_menu

# =========================
# PAGE CONFIGURATION
# =========================
# Page icon (you can add your own logo)
try:
    icon = Image.open(r"c:\Users\susmi\Downloads\AQI.jpeg")  # Add your logo file
except:
    icon = None

# Page configuration
st.set_page_config(
    page_title="Prediction of Air Pollution Using Machine Learning",
    page_icon=icon,
    layout="wide",
    initial_sidebar_state="expanded",
)

# =========================
# CUSTOM CSS STYLING
# =========================
st.markdown("""
    <style>
    /* Header styling */
    .header-title {
        font-size: 35px;
        font-weight: medium;
        color: #000080;
        text-align: center;
        margin-bottom: 10px;
    }
    .subheader-title {
        font-size: 24px;
        font-weight: medium;
        color: #BDB76B;
        text-align: center;
        margin-bottom: 30px;
    }
    
    /* Card styling */
    .card {
        border: 1px solid #ddd;
        border-radius: 8px;
        padding: 16px;
        margin: 8px 0;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        background-color: white;
    }
    .card h3 {
        margin: 0;
        font-size: 18px;
        color: #333;
    }
    .card p {
        margin: 4px 0;
        font-size: 14px;
        color: #666;
    }
    
    /* Button styling */
    .stButton > button {
        background-color: #4CAF50;
        color: white;
        border: none;
        border-radius: 4px;
        padding: 10px 20px;
        cursor: pointer;
        font-weight: bold;
    }
    .stButton > button:hover {
        background-color: #45a049;
    }
    
    /* Reset button styling */
    .reset-btn {
        background-color: #ff4444 !important;
        color: white !important;
    }
    .reset-btn:hover {
        background-color: #cc0000 !important;
    }
    
    /* Input field styling */
    .stNumberInput > div > div > input {
        border-radius: 4px;
        border: 1px solid #ddd;
    }
    
    /* Project title styling */
    .project-title {
        font-size: 40px;
        font-weight: bold;
        text-align: center;
        color: #000080;
        margin-bottom: 10px;
        text-transform: uppercase;
    }
    .project-subtitle {
        font-size: 28px;
        font-weight: bold;
        text-align: center;
        color: #4CAF50;
        margin-bottom: 30px;
    }
    </style>
""", unsafe_allow_html=True)

# =========================
# HEADER SECTION
# =========================
st.title("Yenugu Susmitha Reddy")
st.subheader("Prediction of Air Pollution Using Machine Learning")


# =========================
# SIDEBAR NAVIGATION
# =========================
with st.sidebar:
    if icon:
        st.sidebar.image(icon, use_container_width=True)
    
    selected = option_menu(
        menu_title="🌍 Navigation",
        options=["Home", "AQI Prediction", "Historical Data", "City Analysis", "About"],
        icons=["house", "speedometer2", "clock-history", "building", "info-circle"],
        menu_icon="cast",
        default_index=0,
        styles={
            "container": {"padding": "5px", "background-color": "#f8f9fa"},
            "icon": {"color": "orange", "font-size": "18px"},
            "nav-link": {
                "font-size": "16px",
                "text-align": "left",
                "margin": "5px",
                "--hover-color": "#e9ecef"
            },
            "nav-link-selected": {"background-color": "#4CAF50", "color": "white"},
        }
    )

# Add balloons effect for welcome
if selected == "Home":
    st.balloons()

# =========================
# DATA LOADING FUNCTIONS
# =========================
@st.cache_data
def load_model():
    try:
        with open("air_pollution_model.pkl", "rb") as file:
            return pickle.load(file)
    except:
        st.error("Model file not found. Please train the model first.")
        return None

@st.cache_data
def load_visualization_data():
    try:
        data = pd.read_csv("aqi_visualization_data.csv", parse_dates=['Date'])
        return data
    except:
        st.warning("Historical data not available. Run data processing script first.")
        return None

@st.cache_data
def load_pollutant_stats():
    try:
        with open('pollutant_statistics.json', 'r') as f:
            return json.load(f)
    except:
        return None

@st.cache_data
def load_example_scenarios():
    try:
        with open('example_scenarios.json', 'r') as f:
            scenarios = json.load(f)
            # Remove Typical Hyderabad Day and Typical Bangalore Day
            if 'Typical Hyderabad Day' in scenarios:
                del scenarios['Typical Hyderabad Day']
            if 'Typical Bangalore Day' in scenarios:
                del scenarios['Typical Bangalore Day']
            return scenarios
    except:
        return {
            'Clean Air Day': {'CO': 2.0, 'Ozone': 25.0, 'PM10': 15.0, 'PM25': 10.0, 'NO2': 15.0},
            'Moderate Pollution': {'CO': 5.0, 'Ozone': 50.0, 'PM10': 35.0, 'PM25': 25.0, 'NO2': 30.0},
            'High Pollution': {'CO': 10.0, 'Ozone': 80.0, 'PM10': 60.0, 'PM25': 50.0, 'NO2': 60.0}
        }

# =========================
# HOME PAGE
# =========================
if selected == "Home":
    st.title("🌤️ Welcome to Air Pollution Prediction System")
    st.markdown("---")
    
    # Introduction cards
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.markdown("""
        <div class="card">
            <h3>🤖 ML-based Prediction</h3>
            <p>Predict Air Quality Index using advanced machine learning algorithms</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col2:
        st.markdown("""
        <div class="card">
            <h3>📈 Historical Analysis</h3>
            <p>Explore air pollution trends from 2020-2025 across multiple cities</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col3:
        st.markdown("""
        <div class="card">
            <h3>🏙️ City Comparison</h3>
            <p>Compare air pollution levels between different Indian cities</p>
        </div>
        """, unsafe_allow_html=True)
    
    st.markdown("---")
    
    # System Overview
    st.subheader("📋 System Overview")
    st.markdown("""
    This system **"Prediction of Air Pollution Using Machine Learning"** is designed to monitor, analyze, 
    and forecast air quality levels using advanced machine learning techniques. The system leverages 
    historical air quality data to predict Air Quality Index (AQI) and provides actionable insights 
    for environmental management and public health protection.
    
    ### Key Features:
    - **Real-time AQI Prediction**: Input pollutant values to get instant AQI predictions
    - **Historical Data Analysis**: Explore 5 years of comprehensive air quality data
    - **City-wise Comparison**: Compare pollution levels across multiple cities
    - **Interactive Visualizations**: Dynamic charts and graphs for better understanding
    - **Educational Scenarios**: Learn about different pollution scenarios
    """)
    
    # How to Use
    st.markdown("---")
    st.subheader("🚀 How to Use This System")
    
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.markdown("""
        ### 1. AQI Prediction
        - Go to **AQI Prediction** tab
        - Enter pollutant values manually
        - Or select from predefined scenarios
        - Click 'Predict AQI' to get results
        """)
    
    with col2:
        st.markdown("""
        ### 2. Historical Analysis
        - Navigate to **Historical Data** tab
        - Select date range and cities
        - Choose pollutant to analyze
        - Click 'Apply Filters & Analyze'
        """)
    
    with col3:
        st.markdown("""
        ### 3. City Analysis
        - Go to **City Analysis** tab
        - Select a city from dropdown
        - View detailed pollution statistics
        - Analyze trends and patterns
        """)

# =========================
# AQI PREDICTION PAGE
# =========================
elif selected == "AQI Prediction":
    st.title("📊 Air Pollution Prediction Dashboard")
    
    # Load model and data
    model = load_model()
    pollutant_stats = load_pollutant_stats()
    example_scenarios = load_example_scenarios()
    
    if model is None:
        st.error("Model not available. Please train the model first.")
        st.stop()
    
    # Create tabs like Price Tracker
    tab1, tab2, tab3 = st.tabs(["Manual Input", "Quick Scenarios", "Guidance"])
    
    with tab1:
        st.subheader("📝 Enter Pollutant Values")
        
        # Two-column layout
        col1, col2 = st.columns(2)
        
        with col1:
            # CO input - DIRECT NUMBER INPUT (not slider)
            st.write("**CO (Carbon Monoxide)**")
            st.write("Units: ppm (parts per million)")
            co = st.number_input(
                "Enter CO value:",
                min_value=0.0,
                max_value=100.0,
                value=5.0,
                step=0.1,
                key="co_input"
            )
            
            # Ozone input - DIRECT NUMBER INPUT
            st.write("**Ozone (O3)**")
            st.write("Units: ppb (parts per billion)")
            o3 = st.number_input(
                "Enter Ozone value:",
                min_value=0.0,
                max_value=300.0,
                value=30.0,
                step=0.5,
                key="o3_input"
            )
        
        with col2:
            # PM10 input - DIRECT NUMBER INPUT
            st.write("**PM10**")
            st.write("Units: μg/m³ (micrograms per cubic meter)")
            pm10 = st.number_input(
                "Enter PM10 value:",
                min_value=0.0,
                max_value=500.0,
                value=15.0,
                step=1.0,
                key="pm10_input"
            )
            
            # PM2.5 input - DIRECT NUMBER INPUT
            st.write("**PM2.5**")
            st.write("Units: μg/m³ (micrograms per cubic meter)")
            pm25 = st.number_input(
                "Enter PM2.5 value:",
                min_value=0.0,
                max_value=500.0,
                value=25.0,
                step=1.0,
                key="pm25_input"
            )
            
            # NO2 input - DIRECT NUMBER INPUT
            st.write("**NO2 (Nitrogen Dioxide)**")
            st.write("Units: ppb (parts per billion)")
            no2 = st.number_input(
                "Enter NO2 value:",
                min_value=0.0,
                max_value=200.0,
                value=20.0,
                step=0.5,
                key="no2_input"
            )
    
    with tab2:
        st.subheader("🎯 Quick Scenario Selection")
        
        if example_scenarios:
            scenario_names = list(example_scenarios.keys())
            selected_scenario = st.selectbox("Choose a scenario:", scenario_names)
            
            if selected_scenario:
                scenario = example_scenarios[selected_scenario]
                
                col1, col2 = st.columns([2, 1])
                with col1:
                    st.info(f"**{selected_scenario}**")
                    # Apply button
                    if st.button("Apply This Scenario", type="primary", key="apply_scenario"):
                        st.session_state.co = scenario.get('CO', 5.0)
                        st.session_state.o3 = scenario.get('Ozone', 30.0)
                        st.session_state.pm10 = scenario.get('PM10', 15.0)
                        st.session_state.pm25 = scenario.get('PM25', 25.0)
                        st.session_state.no2 = scenario.get('NO2', 20.0)
                        st.success("Scenario applied! Switch to Manual Input tab.")
                
                with col2:
                    # Show scenario values
                    scenario_df = pd.DataFrame({
                        'Pollutant': ['CO', 'Ozone', 'PM10', 'PM2.5', 'NO2'],
                        'Value': [
                            scenario.get('CO', 5.0), scenario.get('Ozone', 30.0),
                            scenario.get('PM10', 15.0), scenario.get('PM25', 25.0),
                            scenario.get('NO2', 20.0)
                        ]
                    })
                    st.dataframe(scenario_df, use_container_width=True)
    
    with tab3:
        st.subheader("ℹ️ Input Value Guidance")
        st.markdown("""
        ### Understanding Pollutant Units:
        
        **1. CO (Carbon Monoxide)** - Measured in ppm
        - **0-5 ppm**: Good air quality
        - **5-10 ppm**: Moderate pollution
        - **10+ ppm**: High pollution
        
        **2. Ozone (O3)** - Measured in ppb
        - **0-50 ppb**: Good
        - **50-100 ppb**: Moderate
        - **100+ ppb**: Unhealthy
        
        **3. PM10** - Particulate Matter ≤10μm (μg/m³)
        - **0-50 μg/m³**: Good
        - **50-100 μg/m³**: Moderate
        - **100+ μg/m³**: Unhealthy
        
        **4. PM2.5** - Fine Particulate Matter (μg/m³)
        - **0-25 μg/m³**: Good
        - **25-50 μg/m³**: Moderate
        - **50+ μg/m³**: Unhealthy
        
        **5. NO2** - Nitrogen Dioxide (ppb)
        - **0-40 ppb**: Good
        - **40-80 ppb**: Moderate
        - **80+ ppb**: Unhealthy
        """)
    
    # Prediction button and Reset button
    st.markdown("---")
    predict_col1, predict_col2, predict_col3 = st.columns([2, 1, 1])
    
    with predict_col1:
        st.markdown("### 🚀 Ready to Predict")
    
    with predict_col2:
        if st.button("Predict AQI", type="primary", use_container_width=True):
            # Get input values
            co_val = st.session_state.get('co', co)
            o3_val = st.session_state.get('o3', o3)
            pm10_val = st.session_state.get('pm10', pm10)
            pm25_val = st.session_state.get('pm25', pm25)
            no2_val = st.session_state.get('no2', no2)
            
            # Make prediction
            try:
                input_data = np.array([[co_val, o3_val, pm10_val, pm25_val, no2_val]])
                prediction = model.predict(input_data)
                aqi_value = int(prediction[0])
                
                # Store in session
                st.session_state.prediction = aqi_value
                st.session_state.show_result = True
            except Exception as e:
                st.error(f"Prediction error: {e}")
    
    with predict_col3:
        # RESET BUTTON for AQI Prediction page only
        if st.button("🔄 Reset", key="reset_prediction", use_container_width=True, 
                    type="secondary"):
            # Clear session state for this page
            keys_to_clear = ['co', 'o3', 'pm10', 'pm25', 'no2', 'prediction', 'show_result']
            for key in keys_to_clear:
                if key in st.session_state:
                    del st.session_state[key]
            st.success("Inputs reset! Enter new values.")
            st.rerun()
    
    # Show prediction result
    if st.session_state.get('show_result', False):
        aqi_value = st.session_state.prediction
        
        # AQI Categories
        if aqi_value <= 50:
            category = "Good"
            color = "#00E400"
            icon = "😊"
            advice = "Air quality is satisfactory"
        elif aqi_value <= 100:
            category = "Moderate"
            color = "#FFFF00"
            icon = "😐"
            advice = "Acceptable air quality"
        elif aqi_value <= 150:
            category = "Unhealthy for Sensitive Groups"
            color = "#FF7E00"
            icon = "😷"
            advice = "Sensitive groups should take caution"
        elif aqi_value <= 200:
            category = "Unhealthy"
            color = "#FF0000"
            icon = "😟"
            advice = "Everyone may be affected"
        elif aqi_value <= 300:
            category = "Very Unhealthy"
            color = "#8F3F97"
            icon = "🚨"
            advice = "Health alert"
        else:
            category = "Hazardous"
            color = "#7E0023"
            icon = "⚠️"
            advice = "Emergency conditions"
        
        # Display result in card
        st.markdown(f"""
        <div class="card" style="border-left: 10px solid {color};">
            <div style="text-align: center;">
                <h1 style="color: {color}; margin: 0;">{icon} AQI: {aqi_value}</h1>
                <h3 style="color: {color}; margin: 10px 0;">{category}</h3>
                <p style="font-size: 16px;">{advice}</p>
            </div>
        </div>
        """, unsafe_allow_html=True)

# =========================
# HISTORICAL DATA PAGE
# =========================

elif selected == "Historical Data":
    st.title("📊 Historical Data Explorer")
    
    viz_data = load_visualization_data()
    
    if viz_data is not None:
        # Initialize session state for filters if not exists
        if 'filters_applied' not in st.session_state:
            st.session_state.filters_applied = False
        
        # Get min and max dates from data
        min_date = viz_data['Date'].min().date()
        max_date = viz_data['Date'].max().date()
        
        # Use session state values or defaults
        from_date_value = st.session_state.get('from_date_value', min_date)
        to_date_value = st.session_state.get('to_date_value', max_date)
        
        # Filters section
        st.subheader("🔍 Filter Options")
        
        col1, col2, col3, col4 = st.columns(4)
        
        with col1:
            from_date = st.date_input("From:", value=from_date_value, 
                                     min_value=min_date, max_value=max_date,
                                     key="from_date_widget")
        
        with col2:
            to_date = st.date_input("To:", value=to_date_value,
                                   min_value=min_date, max_value=max_date,
                                   key="to_date_widget")
        
        with col3:
            if 'Site Name (of Overall AQI)' in viz_data.columns:
                cities = sorted(viz_data['Site Name (of Overall AQI)'].unique())
                # Get selected cities from session state or default to empty list
                default_cities = st.session_state.get('selected_cities_value', [])
                selected_cities = st.multiselect("Select Cities:", cities, 
                                                default=default_cities,
                                                key="cities_widget")
        
        with col4:
            pollutant_options = ['CO', 'Ozone', 'PM10', 'PM25', 'NO2', 'Overall AQI Value']
            default_pollutant = st.session_state.get('selected_pollutant_value', 'Overall AQI Value')
            selected_pollutant = st.selectbox("Select Pollutant:", pollutant_options, 
                                             index=pollutant_options.index(default_pollutant) if default_pollutant in pollutant_options else 0,
                                             key="pollutant_widget")
        
        # Apply button and Reset button
        st.markdown("---")
        col1, col2, col3 = st.columns([2, 1, 1])
        
        with col1:
            apply_pressed = st.button("Apply Filters & Analyze", type="primary", use_container_width=True)
        
        with col2:
            reset_pressed = st.button("🔄 Reset Filters", key="reset_history", 
                                     use_container_width=True, type="secondary")
        
        # Handle Apply button
        if apply_pressed:
            # Store filter values in session state (with different names to avoid conflict)
            st.session_state.from_date_value = from_date
            st.session_state.to_date_value = to_date
            st.session_state.selected_cities_value = selected_cities
            st.session_state.selected_pollutant_value = selected_pollutant
            st.session_state.filters_applied = True
            st.rerun()
        
        # Handle Reset button
        if reset_pressed:
            # Clear filter values from session state
            keys_to_clear = ['filters_applied', 'from_date_value', 'to_date_value', 
                            'selected_cities_value', 'selected_pollutant_value']
            for key in keys_to_clear:
                if key in st.session_state:
                    del st.session_state[key]
            st.success("Filters reset!")
            st.rerun()
        
        # Apply filters if set
        filtered_data = viz_data.copy()
        
        if st.session_state.get('filters_applied', False):
            # Get values from session state
            from_date_val = st.session_state.get('from_date_value')
            to_date_val = st.session_state.get('to_date_value')
            selected_cities_val = st.session_state.get('selected_cities_value', [])
            selected_pollutant_val = st.session_state.get('selected_pollutant_value', 'Overall AQI Value')
            
            if from_date_val and to_date_val:
                filtered_data = filtered_data[
                    (filtered_data['Date'].dt.date >= from_date_val) & 
                    (filtered_data['Date'].dt.date <= to_date_val)
                ]
            
            if selected_cities_val:
                filtered_data = filtered_data[filtered_data['Site Name (of Overall AQI)'].isin(selected_cities_val)]
            
            # Display results
            st.markdown(f"**Showing:** {len(filtered_data):,} records")
            
            if len(filtered_data) == 0:
                st.warning("No data found with the selected filters. Try different filters.")
            else:
                # Tabs - Now 4 tabs with City-wise AQI added
                tab1, tab2, tab3, tab4 = st.tabs(["Time Trends", "City Comparison", "City-wise AQI", "Statistics"])
                
                with tab1:
                    st.subheader("📈 Time Series Analysis")
                    
                    fig = px.line(filtered_data.sort_values('Date'), x='Date', y=selected_pollutant_val,
                                color='Site Name (of Overall AQI)' if 'Site Name (of Overall AQI)' in filtered_data.columns else None,
                                title=f'{selected_pollutant_val} Over Time')
                    st.plotly_chart(fig, use_container_width=True)
                
                with tab2:
                    st.subheader("🌍 City Comparison")
                    
                    if 'Site Name (of Overall AQI)' in filtered_data.columns:
                        city_stats = filtered_data.groupby('Site Name (of Overall AQI)')[selected_pollutant_val].agg(['mean', 'min', 'max']).round(2)
                        st.dataframe(city_stats, use_container_width=True)
                        
                        fig = px.bar(city_stats.reset_index(), x='Site Name (of Overall AQI)', y='mean',
                                    title=f'Average {selected_pollutant_val} by City')
                        st.plotly_chart(fig, use_container_width=True)
                
                with tab3:
                    st.subheader("🏙️ City-wise AQI Distribution")
                    
                    # City-wise AQI average
                    city_avg = filtered_data.groupby('Site Name (of Overall AQI)')['Overall AQI Value'].mean().reset_index()
                    
                    # Create bar chart
                    fig = px.bar(city_avg, x='Site Name (of Overall AQI)', y='Overall AQI Value',
                                title='Average AQI by City', 
                                color='Site Name (of Overall AQI)',
                                text='Overall AQI Value',
                                color_discrete_sequence=px.colors.qualitative.Set2)
                    fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')
                    fig.update_layout(xaxis_title="City", yaxis_title="Average AQI")
                    st.plotly_chart(fig, use_container_width=True)
                    
                    # Display city statistics
                    st.subheader("City-wise AQI Statistics")
                    col1, col2, col3, col4 = st.columns(4)
                    
                    for idx, city in enumerate(city_avg['Site Name (of Overall AQI)'].unique()):
                        city_data = filtered_data[filtered_data['Site Name (of Overall AQI)'] == city]
                        with col1 if idx % 4 == 0 else col2 if idx % 4 == 1 else col3 if idx % 4 == 2 else col4:
                            avg_aqi = city_data['Overall AQI Value'].mean()
                            st.metric(f"{city} AQI", f"{avg_aqi:.1f}")
                
                with tab4:
                    st.subheader("📊 Statistical Analysis")
                    
                    col1, col2, col3, col4 = st.columns(4)
                    with col1:
                        st.metric("Average", f"{filtered_data[selected_pollutant_val].mean():.2f}")
                    with col2:
                        st.metric("Median", f"{filtered_data[selected_pollutant_val].median():.2f}")
                    with col3:
                        st.metric("Minimum", f"{filtered_data[selected_pollutant_val].min():.2f}")
                    with col4:
                        st.metric("Maximum", f"{filtered_data[selected_pollutant_val].max():.2f}")
                    
                    fig = px.histogram(filtered_data, x=selected_pollutant_val, title='Distribution')
                    st.plotly_chart(fig, use_container_width=True)
        else:
            if not reset_pressed:  # Don't show this message when resetting
                st.info("👆 Please select filters and click 'Apply Filters & Analyze' to see the data.")
    
    else:
        st.error("Historical data not available. Please run the data processing script first.")


# =========================
# CITY ANALYSIS PAGE
# =========================
elif selected == "City Analysis":
    st.title("🏙️ City-wise Air Pollution Analysis")
    
    viz_data = load_visualization_data()
    
    if viz_data is not None:
        # City selection with Reset button
        col1, col2 = st.columns([3, 1])
        
        with col1:
            cities = sorted(viz_data['Site Name (of Overall AQI)'].unique())
            selected_city = st.selectbox("Select a City:", cities, key="city_select")
        
        with col2:
            # RESET BUTTON for City Analysis page
            if st.button("🔄 Reset", key="reset_city", use_container_width=True, 
                        type="secondary"):
                keys_to_clear = ['city_select']
                for key in keys_to_clear:
                    if key in st.session_state:
                        del st.session_state[key]
                st.success("City selection reset!")
                st.rerun()
        
        if selected_city:
            city_data = viz_data[viz_data['Site Name (of Overall AQI)'] == selected_city]
            
            # City metrics
            col1, col2, col3, col4 = st.columns(4)
            with col1:
                st.metric("Average AQI", f"{city_data['Overall AQI Value'].mean():.1f}")
            with col2:
                st.metric("Best AQI", f"{city_data['Overall AQI Value'].min():.1f}")
            with col3:
                st.metric("Worst AQI", f"{city_data['Overall AQI Value'].max():.1f}")
            with col4:
                st.metric("Records", f"{len(city_data):,}")
            
            # Time series for selected city
            fig = px.line(city_data.sort_values('Date'), x='Date', y='Overall AQI Value',
                         title=f'AQI Trend in {selected_city}')
            st.plotly_chart(fig, use_container_width=True)
            
            # Monthly patterns
            city_data['Month-Year'] = city_data['Date'].dt.strftime('%b %Y')
            monthly_avg = city_data.groupby('Month-Year')['Overall AQI Value'].mean().reset_index()
            
            fig2 = px.bar(monthly_avg, x='Month-Year', y='Overall AQI Value',
                         title=f'Monthly Average AQI in {selected_city}')
            st.plotly_chart(fig2, use_container_width=True)
            
            # Pollutant analysis for the city
            st.subheader("📊 Pollutant Analysis")
            pollutant_cols = ['CO', 'Ozone', 'PM10', 'PM25', 'NO2']
            
            pollutant_avg = city_data[pollutant_cols].mean().reset_index()
            pollutant_avg.columns = ['Pollutant', 'Average']
            
            fig3 = px.bar(pollutant_avg, x='Pollutant', y='Average',
                         title=f'Average Pollutant Levels in {selected_city}')
            st.plotly_chart(fig3, use_container_width=True)

# =========================
# ABOUT PAGE
# =========================
elif selected == "About":
    st.title("About Air Pollution Prediction System")
    
    col1, col2 = st.columns([3, 1])
    
    with col1:
        st.markdown("""
        ## 🌍 Project Overview
        
        This comprehensive system **"Prediction of Air Pollution Using Machine Learning"** is designed to 
        monitor, analyze, and forecast air quality levels using advanced machine learning techniques. 
        The system leverages historical air quality data to predict Air Quality Index (AQI) and provides 
        actionable insights for environmental management and public health protection.
        """)
    
    with col2:
        # Add project logo if available
        try:
            logo = Image.open(r"c:\Users\susmi\Downloads\AQI.jpeg")
            st.image(logo, width=150)
        except:
            st.info("Project Logo")

    st.markdown("---")
    
    # Project Details in Tabs
    tab1, tab2, tab3,  = st.tabs(["🎯 Objectives", "🛠️ Methodology", "📊 Data & Model", ])
    
    with tab1:
        st.subheader("Project Objectives")
        st.markdown("""
        ### Primary Goals of the Project:
        
        1. **Air Pollution Prediction**: Develop an accurate machine learning model to predict 
           Air Quality Index (AQI) based on multiple pollutant parameters
        
        2. **Historical Trend Analysis**: Analyze 5 years of air quality data (2020-2025) 
           to identify pollution patterns and seasonal variations
        
        3. **Multi-city Comparison**: Enable comparative analysis of air pollution levels 
           across different Indian cities
        
        4. **Real-time Assessment**: Provide instant AQI predictions based on user-input 
           pollutant concentrations
        
        5. **Environmental Awareness**: Create an educational platform to increase public 
           awareness about air pollution and its health impacts
        
        """)
    
    with tab2:
        st.subheader("Methodology & Technical Approach")
        st.markdown("""
        ### 🧪 Implementation Methodology:
        
        **1. Data Collection & Preprocessing**
        - Collected comprehensive air quality data from monitoring stations
        - Handled missing values and data inconsistencies
        - Normalized pollutant measurements for machine learning compatibility
        - Created temporal features for time-series analysis
        
        **2. Machine Learning Implementation**
        - Selected Random Forest Regressor for its robustness in regression tasks
        - Used 5 key air pollutants as predictive features
        - Implemented cross-validation to ensure model generalizability
        - Optimized hyperparameters for maximum prediction accuracy
        
        **3. System Architecture**
        - Backend: Python-based machine learning pipeline
        - Frontend: Streamlit web application framework
        - Database: Processed CSV files with historical air quality data
        - Visualization: Interactive plots using Plotly and Matplotlib
        
        **4. Model Deployment**
        - Serialized trained model using Pickle
        - Created RESTful prediction endpoints
        - Implemented user-friendly interface with real-time feedback
        - Added scenario simulation for educational purposes
        """)
    
    with tab3:
        st.subheader("Data & Model Specifications")
        
        col1, col2 = st.columns(2)
        
        with col1:
            st.markdown("""
            ### 📈 Air Quality Dataset:
            
            **Source**: Real-time monitoring stations across India
            
            **Temporal Coverage**: January 2020 - December 2025
            
            **Geographical Coverage**:
            - Hyderabad (Telangana)
            - Bangalore (Karnataka)
            - Delhi (National Capital Region)
            - Visakhapatnam (Andhra Pradesh)
            
            **Pollutant Parameters**:
            1. **CO (Carbon Monoxide)** - Measured in ppm (parts per million)
            2. **Ozone (O₃)** - Measured in ppb (parts per billion)
            3. **PM10** - Particulate Matter ≤10μm (μg/m³)
            4. **PM2.5** - Fine Particulate Matter ≤2.5μm (μg/m³)
            5. **NO₂** - Nitrogen Dioxide (ppb)
            
            **Target Variable**: Overall AQI Value
            """)
        
        with col2:
            st.markdown("""
            ### 🤖 Machine Learning Model:
            
            **Algorithm**: Random Forest Regressor
            
            **Input Features**: 5 pollutant concentrations
            
            **Output**: Predicted AQI Value
            
            **Model Performance Metrics**:
            - R² Score (Coefficient of Determination): **> 0.85**
            - Mean Absolute Error (MAE): **< 15 AQI points**
            - Root Mean Square Error (RMSE): **< 20 AQI points**
            
            **Data Split**:
            - Training Data: **80%** (Model development)
            - Testing Data: **20%** (Performance evaluation)
            
            **Feature Importance**:
            - PM2.5 and PM10 identified as most significant predictors
            - All 5 pollutants contribute to AQI prediction
            """)

         
    
    
    # Key Features Section
    st.subheader("✨ System Features & Capabilities")
    
    features = [
        {"icon": "🤖", "title": "ML-based Prediction", "desc": "Accurate AQI prediction using Random Forest algorithm"},
        {"icon": "📊", "title": "Real-time Analysis", "desc": "Instant AQI calculation based on pollutant inputs"},
        {"icon": "📈", "title": "Historical Data Explorer", "desc": "5-year comprehensive air quality data analysis"},
        {"icon": "🏙️", "title": "City Comparison", "desc": "Compare pollution levels across 4 major Indian cities"},
        {"icon": "🌫️", "title": "Pollutant Contribution", "desc": "Analyze individual pollutant impact on AQI"},
        {"icon": "🎯", "title": "Scenario Simulation", "desc": "Test various pollution scenarios and their AQI impact"},
        {"icon": "📱", "title": "User-friendly Interface", "desc": "Intuitive design with easy navigation"},
        {"icon": "📋", "title": "Comprehensive Reports", "desc": "Detailed statistical analysis and visualizations"}
    ]
    
    # Display features in 4 columns
    cols = st.columns(4)
    for i, feature in enumerate(features):
        with cols[i % 4]:
            st.markdown(f"""
            <div class="card" style="height: 200px; margin-bottom: 15px;">
                <div style="font-size: 28px; margin-bottom: 10px; text-align: center;">{feature['icon']}</div>
                <h4 style="margin: 5px 0; text-align: center; color: #000080;">{feature['title']}</h4>
                <p style="font-size: 13px; color: #666; text-align: center;">{feature['desc']}</p>
            </div>
            """, unsafe_allow_html=True)
    
    st.markdown("---")
    
    # Technical Stack
    st.subheader("🛠️ Technology Stack Used")

    st.markdown("""
    - **Programming:** Python 3.11 – Core development language  
    - **ML Framework:** Scikit-learn – Machine learning algorithms  
    - **Web Framework:** Streamlit – Interactive web application  
    - **Data Processing:** Pandas, NumPy – Data manipulation and analysis  
    - **Visualization:** Plotly, Matplotlib – Data plotting and charts  
    - **Model Storage:** Pickle – Model serialization  
    - **Data Storage:** JSON, CSV – Configuration and data files  
    - **Development:** Jupyter Notebook, VS Code – Development environment  
    """)


    
    st.markdown("---")
    
    # Applications & Impact
    st.subheader("📋 Practical Applications & Impact")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.markdown("""
        ### 🏢 Real-world Applications:
        
        **Environmental Monitoring**
        - Continuous air quality tracking
        - Pollution source identification
        - Environmental compliance monitoring
        
        **Public Health Protection**
        - Daily air quality advisories
        - Sensitive group alerts (asthma, elderly)
        - Outdoor activity recommendations
        
        **Educational Tool**
        - Environmental science education
        - Public awareness campaigns
        - Research and academic projects
        
        """)
    
    with col2:
        st.markdown("""
        ### 🌱 Environmental & Social Impact:
        
        **Awareness Generation**
        - Educates public about air pollution dangers
        - Promotes environmental consciousness
        - Encourages sustainable practices
        
        **Health Benefits**
        - Helps prevent respiratory diseases
        - Reduces healthcare burden
        - Improves quality of life
        
        **Economic Impact**
        - Supports tourism industry
        - Attracts clean industry investments
        - Reduces pollution-related economic losses
        """)
    
    st.markdown("---")
    
    
    
    # Final Footer
    st.subheader("Prediction of Air Pollution Using Machine Learning")


# =========================
# FOOTER
# =========================
st.markdown("---")
st.caption("Prediction of Air Pollution Using Machine Learning ")

Overwriting aqi_app.py
